In [ ]:
import os

import pyburst
from pyburst.config import Config
from pyburst.utils import logger_init

if not os.environ.get('HOME_WAT_FILTERS'):
    pyburst_path = os.path.dirname(os.path.abspath(pyburst.__file__))
    os.environ['HOME_WAT_FILTERS'] = f"{os.path.abspath(pyburst_path)}/vendor"

logger_init()

config = Config('./user_parameters_injection.yaml')
config.ifo, config.injection

## generate injected data for each detector with given parameters in config

In [ ]:
from pyburst.modules.read_data import generate_injection
from pyburst.modules.job_segment import create_job_segment_from_injection

job_segments = create_job_segment_from_injection(config.simulation, config.injection)

data = generate_injection(config, job_segments[0])
data

In [ ]:
# plot the pycbc time series for L1
data[0].plot()

## apply data conditioning to the data

In [ ]:
from pyburst.modules.data_conditioning import data_conditioning
from pyburst.modules.plot import plot_spectrogram

strains, nRMS = data_conditioning(config, data)

# plot the spectrogram for L1
plt = plot_spectrogram(strains[0], gwpy_plot=True)
ax = plt.gca()
ax.set_ylim(15,1024)

strains, nRMS

## calculate coherence

In [ ]:
from pyburst.modules.coherence import coherence, sparse_table_from_fragment_clusters

# calculate coherence
fragment_clusters = coherence(config, strains, nRMS)


## supercluster

In [ ]:
from pyburst.modules.super_cluster import supercluster
from pyburst.types import WDMXTalkCatalog, Network

network = Network(config, strains, nRMS)

pwc_list = supercluster(config, network, fragment_clusters, strains)

## likelihood

In [ ]:
from pyburst.modules.likelihood import likelihood

events, clusters = likelihood(config, network, pwc_list)

In [ ]:
from pyburst.modules.plot import plot_event_on_spectrogram

for i, tf_map in enumerate(strains):
    plt = plot_event_on_spectrogram(tf_map, events)
    plt.show()

## plot statistics

In [ ]:
from gwpy.spectrogram import Spectrogram

for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("likelihood")

    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()

In [ ]:
from gwpy.spectrogram import Spectrogram

for cluster in clusters:
    merged_map, start, dt, df = cluster.get_sparse_map("null")

    plt = Spectrogram(merged_map, t0=start, dt=dt, f0=0, df=df).plot()
    plt.colorbar()